# 8. Object References, mutability and recycling
- Object Identity, value and aliasing(객체의 정체성, 동일성, 별명)
- Shallow and deep copies
- References and function parameter
    - Problem with mutable parameter defaults
    - Safe handling of mutable arguments passed by clients of our fuction
- Garbage collection, del, weak reference

## 8.1 Variables are not boxes

In [1]:
a = [1,2,3]
b=a
a.append(4)
print(a, b)

[1, 2, 3, 4] [1, 2, 3, 4]


- 위 예제에서 [1,2,3]이라는 list 객체가 a에 할당되었고, b는 동일한 객체에 할당되었다. 즉, [1,2,3]이라는 2개의 alias(별명)을 갖게 된다.
- a.append(4)는 a가 참조하는 주소에 있는 [1,2,3]에 직접 접근하여, 4라는 원소를 추가. b는 동일한 주소를 refer하기 때문에, b도 [1,2,3,4]라는 결과가 나옴

In [2]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: {}'.format(id(self)))

In [5]:
x = Gizmo()
y = Gizmo()*10

Gizmo id: 139851586244680
Gizmo id: 139851586322384


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [7]:
print(dir())

['Gizmo', 'In', 'Out', '_', '_6', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_ih', '_ii', '_iii', '_oh', 'a', 'b', 'exit', 'get_ipython', 'quit', 'x']


- 위와 같은 경우에는 Gizmo()라는 객체가 먼저 생성되고, x는 이 객체에 할당되었다. 즉, 객체가 먼저 생성되고, 변수가 객체에 할당된다.
- y의 경우 객체와 10간의 operation이 불가능 --> y라는 변수가 생성되지 않음

## 8.2 Identity, equality, aliases(정체성, 동질성, 별명)

In [9]:
#ex 8.3.1
Charles = {'name': 'C.L.Dodgson', 'born': 1832} 
Lewis = Charles #
print(Lewis is Charles)
print(id(Lewis), id(Charles))
Lewis['balance'] = 950
print(Lewis, Charles)

True
139851586273032 139851586273032
{'balance': 950, 'name': 'C.L.Dodgson', 'born': 1832} {'balance': 950, 'name': 'C.L.Dodgson', 'born': 1832}


In [10]:
# ex 8.3.2
Charles = {'name': 'C.L.Dodgson', 'born': 1832}
Lewis = dict(Charles)
print(Lewis is Charles)
print(id(Lewis), id(Charles))
Lewis['balance'] = 950
print(Lewis, Charles)

False
139851585768776 139851585767816
{'balance': 950, 'name': 'C.L.Dodgson', 'born': 1832} {'name': 'C.L.Dodgson', 'born': 1832}


- 8.3.1의 경우, dict의 object가 생성되고, Charles라는 변수를 이 object에 할당했다. 또한, Lewis라는 변수도 이 object에 할당하였다. 동일한 객체가 두개의 alias를 갖는다.
- 따라서 두 변수의 정체성은 같다. 그러므로, Lewis가 갖고있는 주소값에 해당하는 dict object에 접근하여 직접 값을 바꾸면, 같은 주소값을 갖고있는 Charles도 변하게 되는것이다.
- 8.3.2의 경우, dict의 object가 생성되고, dict(Charles)를 하면, Charles에 해당하는 dict object를 shallow copy하는 것. 즉, Lewis와 Charles는 값은 같지만 서로 다른 object에 할당되었음
- is: 객체의 정체성을 비교
- ==: 객체의 값을 비교

In [13]:
# ex 8.4
Alex = {'balance': 950, 'name': 'C.L.Dodgson', 'born': 1832} 
print(Alex == Lewis) # 값 비교
print(Alex is Lewis) # 정체성 비교

True
False


### 8.2.2 The relative immutability of tuples

- 아래 경우에서, int가 생성되고, 이를 저장하면, 다음에 이 값을 부를 때, 해당 주소를 return하는 것을 봤을 때, 실제로 가지고 오는 것은 

In [50]:
# ex 8.5.0
t1 = (1,2,[3,4])
t2 = (1,2,[3,4])
t3 = (1,2,[5,6])

In [51]:
print(id(t1[0]), id(t1[1]), id(t1[-1]))
print(id(t2[0]), id(t2[1]), id(t2[-1]))
print(id(t3[0]), id(t3[1]), id(t3[-1]))

93878511743040 93878511743072 139851585922568
93878511743040 93878511743072 139851585845960
93878511743040 93878511743072 139851585911816


- 아래의 경우, (1,2,[3,4])라는 tuple을 만들고, t1을 할당한다. 마찬가지 방법으로 t2를 할당한다. 이 때, 튜플안의 각 element도 주소값에서 받아옴.
- t1[-1].append(9)는 t1[-1]이 가지고 있는 주소값에 [3,4]라는 list가 저장이 되어 있는데, 여기에 직접 접근하여 append(9)한다.
- 그러나 , t2[-1]에는 다른 주소값이 있기 때문에, [3,4]에는 영향이 없다.

In [49]:
print(t1 == t2, t1 is t2)
print(id(t1[-1]))
t1[-1].append(9)
print(t1, t2)
print(id(t1[-1]))
print(t1 == t2)

False False
139851585911816
(1, 2, [3, 4, 9, 9]) (1, 2, [3, 4])
139851585911816
False


## 8.3 Copies are shallow by default

- copy하는 가장 쉬운 방법은, 그 자료형 자체의 생성자를 사용하는 것 ex) list(list1)

In [53]:
list1 = [3, [44,55], (7,8,9)]
list2 = list(list1)
print(list1 == list2, list1 is list2)

True False


- Shallow copy: outermost만 copy, inner는 원래 들어있던 동일 object에 대한 reference가 됨
- inner에 immutable한 element만 있을 경우, 메모리를 절약하여 문제를 일으키지 않음. 그러나, mutable의 element가 들어 있을 경우, 문제가 발생함

In [59]:
list1 = [3, [44,55], (7,8,9)] #0
list2 = list(list1) #1
list1.append(100) #2
list1[1].remove(55) #3
print(list1, '______________', list2)
print(id(list1[1]), '______________', id(list2[1]))
print(id(list1[2]), '______________', id(list2[2]))
list2[1]+=[66, 77] #4
list2[2]+=(10,11) #5
print(list1, '______________', list2)
print(id(list1[1]), '______________', id(list2[1]))
print(id(list1[2]), '______________',  id(list2[2]))

[3, [44], (7, 8, 9), 100] ______________ [3, [44], (7, 8, 9)]
139851586244232 ______________ 139851586244232
139851586781832 ______________ 139851586781832
[3, [44, 66, 77], (7, 8, 9), 100] ______________ [3, [44, 66, 77], (7, 8, 9, 10, 11)]
139851586244232 ______________ 139851586244232
139851586781832 ______________ 139851586695768


- 위 코드는 다음과 같이 이루어 진다.    
    - 0: list object를 생성하여, list1라는 변수를  이에 할당
    - 1: list1을 shallow copy한 object를 생성하여, list2라는 변수를 이에 할당. 두 object는 reference값이 다르지만, object의 원소들은 동일한 reference값을 갖는다.
    - 2: list1이 갖고 있는 reference 값에 접근하여 직접 100을 append
    - 3: list1[1]이 갖고 있는 reference 값에는 [44, 55]가 저장되어 있었다. 여기에 접근하여 55를 제거
    - 결과적으로, list1, list2가 갖고 있는 reference값이 다르기 때문에, list2는 100이 append 안됨. 그러나, list1[1]과 list2[1]의 refence값은 동일하기 때문에 55가 공동으로 제거
    - 4: list2[1]이 갖고 있는 reference값에 접근하여 66, 77을 추가함
    - 5: tuple은 immutable --> += 연산자는  list2[2] = list2[2] + (10,11) 가 된, 새로운 tuple을 만들고, 새로운 주소에 저장한 뒤, list2[2]를 이에 할당

### 8.3.1 Deep and shallow copies of arbitrary objects

In [62]:
import copy
class Bus:
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)        
    def drop(self, name):
        self.passengers.remove(name)
        
bus1 = Bus(['A','B','C','D'])
bus2 = copy.copy(bus1) # bus2의 attribute와 bus1의 attribute는 같은 reference값을 갖는다.
bus3 = copy.deepcopy(bus1) # bus1의 attribute까지 copy
print(id(bus1), id(bus2), id(bus3))
bus1.drop('A') # bus1과 bus2의 attribute값이 같은 reference값 --> bus1.passengers에서 'A'가 drop --> bus2.passengers에서도 같은 결과값이 나오게 됨
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print(bus3.passengers)

139851585943144 139851587211104 139851585943256
['B', 'C', 'D']
139851585650184 139851585650184 139851585646792
['A', 'B', 'C', 'D']


- deep copies를 만드는 일은 순환 참조가 일어나, 무한 루프에 빠질 가능성이 있다. deepcopy는 순환참조를 제대로 처리하기 위해, 이미 복사한 객체에 대한 참조를 기억한다.

In [65]:
a = [1,2]
b = [a,3]
a.append(b)
print(a)
c = copy.copy(a)
print(c)
d = copy.deepcopy(a)
print(d)

[1, 2, [[...], 3]]
[1, 2, [[1, 2, [...]], 3]]
[1, 2, [[...], 3]]


## 8.4 Function parameters as references

- Python은 callby sharing하는 parameter 전달 방식만 지원. -> 각 parameter가 인수로 전달받은 각 reference의 copy를 받는다는 의미. 즉, parameter가 실제 인수의 별명이 됨
- Function은 인수로 전달받은 모든 mutable object를 변경할 수 있지만, identity를 변경할 수는 없다!

In [68]:
def f(a,b):
    a += b
    return a

x,y = 1,2
print(f(x,y), x,y)
print(id(f(x,y)), id(x))

x, y = [1,2], [3,4]
print(f(x,y), x, y)
print(id(f(x,y)), id(x)) # x가 갖는 주소값에 직접 접근하여, 해당하는 값인 [1,2]를 f(x,y)값으로 바꿈. --> f(x,y), x의 결과와 주소 값이 같음

x, y = (1,2), (3,4)
print(f(x,y), x, y)
print(id(f(x,y)), id(x)) # 새로운 tuple을 만들어서, f(x,y)의 결과 변수를 이에 할당

3 1 2
93878511743104 93878511743040
[1, 2, 3, 4] [1, 2, 3, 4] [3, 4]
139851584574536 139851584574536
(1, 2, 3, 4) (1, 2) (3, 4)
139851585204040 139851586112584


### 8.4.1 Mutable types as parameter defaults: bad idea
- parameter의 default는 매우 좋은 기능 --> 하위 호완성 유지 및 API개선이 가능하게 함
- 그러나, default로 mutable을 사용하는 것은 매우 위험

In [75]:
class HauntedBus:
    def __init__(self, passengers = []):
        self.passengers = passengers
        print(id(self.passengers))
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)
        
bus1 = HauntedBus(['A','B','C','D'])

print('>>>>>Bus1')
print(bus1.passengers)
bus1.pick('E')
bus1.drop('D')
print(bus1.passengers)

print('>>>>>Bus2')
bus2 = HauntedBus() #1
print(bus2.passengers)
bus2.pick('F') #2
print(bus2.passengers)

print('>>>>>Bus3')
bus3 = HauntedBus() #3
print(bus3.passengers)
bus3.pick('G') #4
print(bus2.passengers, bus3.passengers)
print(bus2.passengers is bus3.passengers)
print(bus1.passengers)
print(dir(HauntedBus.__init__))
print(HauntedBus.__init__.__defaults__) # HauntedBus의 기본 값으로 []가 아닌 ['F','G']가 들어가 있다??????
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers) # Hauntedbus의 기본값으로 bus2.passenger의 주소값을 가져옴

139851366035272
>>>>>Bus1
['A', 'B', 'C', 'D']
['A', 'B', 'C', 'E']
>>>>>Bus2
139851366035336
[]
['F']
>>>>>Bus3
139851366035336
['F']
['F', 'G'] ['F', 'G']
True
['A', 'B', 'C', 'E']
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['F', 'G'],)
True


- 문제점: 승객 list가 존재하지 않고, default일 때, bus 객체들이 승객 리스트를 공유 하게 된다!!!
- 이는 다음과 같은 원리로 발생하게 되는 버그이다.
    - HauntedBus class안에서, default값이 없는 bus객체는 self.passengers = passengers로 인하여, passenger의 별명이 self.passenger가 된다.
    - 값이 없는 경우, default passenger의 별명이 self.passenger가 된다. 
    - pick method를 실행할 경우, self.passenger.append()가 실행 되고, 이는 self.passenger가 가리키는 주소인 default passenger에 직접 접근하여 이를 바꿔준다.
    - <b>그런데 문제는 각 기본값은 함수가 정의될 때, 평가 된다. 즉, bus=Bus()할 때, 기본값이 설정 되는 것이 아니고, class를 정의할 때, 이미 설정되는 것이다. </b>
        - 이는 id(bus2.passengers) == id(bus3.passengers)에서 알 수 있다.
    - 그러므로, 기본값이 mutable이고, 이 obj를 변경하면 변경 내용이 영향을 미치는 것이다
    - <b> 이럴 경우, 왠만하면 []말고, None쓰세요!!!</b>

### 8.4.2. Defensive Programming with mutable parameters
- mutable parameter를 받는 함수를 구현할 때, paramter가 변경될 것이라는 것을 caller가 예상할 수 있는가의 여부를 잘 고려해야 한다.
- 함수가 dict obj를 받는다고 하자. 만약 함수내에서 이를 받아 처리하는 동안, dict가 변경된다고 하면... 변경내용은 남아있어야 할까? 아닐까?

In [78]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers=[]
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)
        
team = ['A','B','C','D']
bus = TwilightBus(team)
bus.drop('D')
bus.drop('C')
bus.pick('E')
print(bus.passengers, team)

['A', 'B', 'E'] ['A', 'B', 'E']


- 문제점: self.passenger가 team과 더불어 [A, B, C, D]라는 리스트의 별명. -> self.passengers.append나 self.passengers.remove를 하면, 실제로 self.passengers가 갖고있는 주소에 직접 접근하여 이를 변경함. team도 같은 주소를 갖고 있기 때문에, team을 불러도 변경이 되어버림
- 다음과 같이 고치자!

In [79]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers=[]
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)
        
team = ['A','B','C','D']
bus = TwilightBus(team)
bus.drop('D')
bus.drop('C')
bus.pick('E')
print(bus.passengers, team)

['A', 'B', 'E'] ['A', 'B', 'C', 'D']


## 8.5 del and garbage collection
- del: 객체를 제거하는 것이 아니라, 이름을 제거하는 것!
    - 제거된 변수가 객체를 참조하는 최후의 변수거나 객체에 도달할 수 없을 때만, garbage collect됨
    - 즉, del명령의 결과로 객체가 garbage collect될 수 있을때는 특수한 경우
    - 변수를 다시 binding하더라도, 객체에 대한 reference count를 0으로 만들어, obj가 제거될 수 있음 

In [86]:
import weakref
s1 = {1,2,3} # {1,2,3}에 대한 refer cnt = 1
s2 = s1 # refer cnt = 2

def bye():
    print("AAA")

ender1 = weakref.finalize(s1, bye) # finalize는 {1,2,3}에 대한 weak reference를 갖는다!
ender2 = weakref.finalize(s2, bye)
print(ender1.alive, ender2.alive)
del s1 #refer cnt = 1
print(ender1.alive, ender2.alive)
s2 = {3,4,5} # refer cnt=0
print(ender1.alive, ender2.alive)

ender2 = weakref.finalize(s2, bye)
print(ender2.alive)

AAA
True True
True True
AAA
AAA
False False
True


## 8.6 Weak Reference
- weak reference:  reference count를 증가시키지 않고, 객체를 refer. reference 대상 obj를 referent라고 함
- WeakValueDict: obj에 대한 weak reference를 값으로 가지는 mutable mapping.
    - referent가 프로그램의 다른 곳에서 garbage collect되면, 해당 key도 WeakValueDictionary에서 자동으로 제거됨

In [89]:
import weakref
aset = {0,1}
wref = weakref.ref(aset)
print(wref, wref())
aset={2,3,4}
print(wref())
print(wref() is None)
print(wref() is None)

In [90]:
class Cheese:
    def __init__(self, kind):
        self.kind = kind
    def __repr__(self):
        return ('Cheese(%r)' %self.kind)

import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('A'), Cheese('B'), Cheese('C'), Cheese('D')]
for cheese in catalog:
    stock[cheese.kind] = cheese
print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['A', 'B', 'C', 'D']
['D']
[]


- 위에서 임시변수가 obj를 refer함으로서, 예상보다 obj의 수명이 늘어날 수 있음. local variable은 func의 return값이 생기면서 사라짐. --> 일반적으로 no problem
- 그러나 위의 경우, for문 의 변수인 cheese는 global variable --> 명시적으로 제거하기 전에는 사라지지 않음....????

- Weak reference의 한계점
    - list와 dict obj는 기본적으로 reference 대상이 될수는 없지만, 이들의 subclass에 해당하는 obj들은 다음 코드처럼 쉽게 해결 가능
    - set obj는 reference대상이 될 수 있음.

## 8.7 Tricks Python plays with immutables
- a = (1,2)와 같은 tuple의 경우, a[:]가 copy를 생성하지 않고, 그 obj에 대한 reference를 반환함. tuple(a)로 해도 마찬가지
- str, bytes, frozenset의 경우도 마찬가지

In [92]:
t1 = (1,2,3)
t2 = tuple(t1)
t3 = t1[:]
print(t2 is t1)
print(t3 is t1)

True
True


In [97]:
t1=(1,2,3)
t2=(1,2,3)
t3=tuple(t1)
print(t2 is t1, t3 is t1)

s1 = 'ABC'
s2 = 'ABC'
s3 = str(s1)
print(s2 is s1, s3 is s1)

False True
True True
